In [1]:
import os
import json


# Path Evaluator

In [2]:
# Loading the pathEvaluator json which contain the data regarding the question and choices
with open('./pathEvaluator.json', 'r') as f:
    main_data = json.load(f)

In [3]:
# Question data subset of main_data containing only info about question
question_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["choicePointNavigatorMetadata"]["choicePointsMetadata"]["choicePoints"]  

# choice data subset of main_data containing only info about choices
choice_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["momentsBySegment"]

In [4]:
# bs_data relates parent_id, chocie_id, question, chocice text and default option
bs_data = {}
# stores the choice id and the corresponding text
choice_text = {}
# stores the question id and the corresponding text
# TODO: do it soon
# ques_text = {}

In [5]:
# returns the question for a given patent_id
# uses parent_if to find the question
# requires question data to be declared in the global scope
def find_ques(parent_id):
    return question_data[parent_id]["description"]
    
# example 
# print(find_ques(parent_id="3X"))

In [6]:
# add the relevant data (relates parent_id, chocie_id, question, chocice text and default option) to bs_data from choice_data
# arg : parent_id (id of the previous choice)
# expects question_data and choce_data is present in the global scope
# add the id and corresponding choice text to the choice_text 

def gen_data(parent_id):
    temp = {}
    # Some options may not have any Question 
    # example: single options choice are mostlikely not to have any question
    try:
        temp['question'] = find_ques(parent_id)
    except:
        # do nothing
        pass
        
    data = choice_data[parent_id]
    for items in data:
        if "choices" in items:
            temp["defaultChoiceIndex"] = items["defaultChoiceIndex"]
            # some choices don't have any text
            # example: there is a choice point where the user need to type the phone no 
            # correct phone no will be one choice and the wrong one will be the other.
            try:
                temp["choices"] = [{"id": item["id"], "text": item["text"], "segmentId": item["sg" if "sg" in item else "segmentId"]} for item in items["choices"]]
                
                # adding the choice id and text to choice_text
                for choice in temp["choices"]:
                    if choice["id"] not in choice_text:
                        choice_text[choice["segmentId"]] = choice['text']
            except:
                # do something to add info about the phone no.
                # if ph no is available add else add xxx
                temp["choices"] = [{"id": item["id"], "code": item["code"] if "code" in item else "xxx"} for item in items["choices"]]
                
    bs_data[parent_id]=temp


In [7]:
# returns the default choice for a given parent id
# expects the bs_data (created already) is available in the global scope
# bs_data relates parent_id, chocie_id, question, chocice text and default option

def find_default(parent_id):
    return bs_data[parent_id]["choices"][bs_data[parent_id]["defaultChoiceIndex"]]

    

In [8]:
def get_choice_text(choice_id):
    with open('./choice_text.json', 'r') as f:
        data = json.load(f)
    return(data[choice_id])

In [9]:
# examples

# finds the question and the choice id with text for a given parent id 
# parent id is the id of the preious choice
# find_choices(parent_id="1A")

# print(find_default("1A"))
# find_ques(parent_id="2G")

In [10]:
# storing all id's
parent_ids=choice_data.keys();

In [11]:
# generating data for all the id's
for parent_id in parent_ids:
#     print(parent_id)
    gen_data(parent_id)
    
    
blank_keys = []
for key, data in bs_data.items():
    if len(data) == 0:
        blank_keys.append(key) 
for key in blank_keys:
    del bs_data[key]
len(bs_data)
    
# storing main data
with open('bs_data.json', 'w') as json_file:
  json.dump(bs_data, json_file)

# saving the choice text
with open('choice_text.json', 'w') as json_file:
  json.dump(choice_text, json_file)

# Data cl2

- first, find what is selected
- second, find if it was default or non-default

In [12]:
def decode_cl2(cl2_path="./data_cl2.json"):
    print("Processing : "+cl2_path)
    with open(cl2_path, 'r') as f:
        data_cl2 = json.load(f)
    
    selected_coice_id = ""
    parent_id = ""
    reverseDeltas = data_cl2["reverseDeltas"]
    
    for items in reverseDeltas:
        for item in items:
            if type(item) == dict and "view" in item:
                if item["view"] == "ikoChoicePointOption":
                    selected_coice_id = item["trackingInfo"]["segmentId"]
                elif item["view"] == "ikoChoicePoint":
                    parent_id = item["trackingInfo"]["momentId"].split("_")[1]
    
    if selected_coice_id == "" or parent_id == "":
        print("error: looks Like invalid cl2")
        
    else:
        print('-'*10)
        try:
            print("Selected choice ID :"+ selected_coice_id)
            print("Selected choice :"+get_choice_text(selected_coice_id))
        except:
            print("Opsee! error")
        try:
            print("Question Id: "+parent_id)
            print("Question text: "+find_ques(parent_id))
        except:
            print("No Question present for this choice! ")
            with open('./bs_data.json', 'r') as f:
                bs_data = json.load(f)
            print("This question has {} options".format(len(bs_data[parent_id]["choices"])))

        print("Default chocie: "+str(find_default(parent_id)))
        print('-'*10)
    

In [13]:
decode_cl2("./cl2/data_cl2_0.json")

Processing : ./cl2/data_cl2_0.json
----------
Selected choice ID :1E
Selected choice :SUGAR PUFFS
Question Id: 1A
Question text: Which Cereal?
Default chocie: {'id': '1E', 'text': 'SUGAR PUFFS', 'segmentId': '1E'}
----------


In [16]:
# renameing all files
i = 0
for filename in os.listdir("./cl2/cl2"): 
        dst =str(i) + "_cl2.json"
        src ='./cl2/cl2/'+ filename 
        dst ='./cl2/cl2/'+ dst 
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 
        i += 1

In [17]:
for cl2 in os.listdir("./cl2/cl2"):
    decode_cl2("./cl2/cl2/"+cl2)

Processing : ./cl2/cl2/12_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/10_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/23_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/27_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/8_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/6_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/18_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/26_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/13_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/15_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/22_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/31_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/0_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/17_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/25_cl2.json
error: looks Like invalid cl2
Processing : ./cl2/cl2/3_cl2